In [1]:
from ruptura.RupturaTable import RupturaTable
from ruptura.CreateBatch import CreateBatch
import collections

In [ ]:
rupturaTable = RupturaTable()
files = ['ruptura-2018-dezembro-1oQ.csv', 'ruptura-2018-dezembro-2oQ.csv', 'ruptura-2019-fevereiro.csv', 'ruptura-2019-janeiro-1oQ.csv', 'ruptura-2019-janeiro-2oQ.csv', 'ruptura-2019-marco.csv']
clipData = ['01/11/2018', '01/04/2019']  # mais recente 08/03/2019
rupturaTable.generate(files,clipData)

In [ ]:
createBatch = CreateBatch()
amostras = createBatch.create(rupturaTable.getData(), 'BISCOITO BARBIERI PAPA OVO 150G', '01/03/2019')
#createBatch.exportBatch(amostras)

In [ ]:
from ruptura.CreateBatch import CreateBatch
createBatch = CreateBatch()
amostras = createBatch.loadBatch()
X, Y, Ytest = createBatch.batch(amostras)

In [ ]:
# TODO - consertar a definicao do batch ou talvez transferir para outro objeto.
# TREINO E TESTE DO NOVO CARA - colocar para treinar
# INCORPORAR ESTE DE TODOS OS PRODUTOS
# CRIAR UMA FUNCAO PARA ELIMINAR ESSE QUE TEM INFORMACOES ERRADAS

# MAIS DE UM PRODUTO

In [9]:
referenceDate = '08/03/2019'
data = rupturaTable.getData()
createBatch = CreateBatch()
produtos = list(collections.Counter(data.loc[:,'Produto']))
todosBiscoitos  = []
for prod in produtos:
    if 'BISCOITO BARBIERI' in prod:
        todosBiscoitos.append(prod)
todasAmostras = {}
data = rupturaTable.getData()
for nomeDoProduto in todosBiscoitos:
    amostras = createBatch.create(data, nomeDoProduto, referenceDate)
    todasAmostras.update(dict(amostras))

createBatch.exportBatch(todasAmostras, 'biscoitos-barbieri-8-3-19.json')